In [1]:
import pandas as pd
import numpy as np
import string
from pymystem3 import Mystem

In [95]:
df = pd.read_csv('df_final.csv')

In [96]:
reviews = pd.read_csv('df_reviews.csv')

In [2]:
m = Mystem()

def preprocess(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    lemmas = m.lemmatize(text)
    text = (''.join(lemmas)).strip()
    return text

In [3]:
reviews = 0

cheap = pd.read_csv('./dataset/cheap.csv')
cheap = cheap['0']
cheap = cheap.apply(lambda x: preprocess(str(x)))
reviews += len(cheap)

expensive = pd.read_csv('./dataset/expensive.csv')
expensive = expensive['0']
expensive = expensive.apply(lambda x: preprocess(str(x)))
reviews += len(expensive)

family = pd.read_csv('./dataset/family.csv')
family = family['0']
family = family.apply(lambda x: preprocess(str(x)))
reviews += len(family)

friends = pd.read_csv('./dataset/friends.csv')
friends = friends['0']
friends = friends.apply(lambda x: preprocess(str(x)))
reviews += len(friends)

kids = pd.read_csv('./dataset/kids.csv')
kids = kids['0']
kids = friends.apply(lambda x: preprocess(str(x)))
reviews += len(kids)

romantic = pd.read_csv('./dataset/romantic.csv')
romantic = romantic['0']
romantic = romantic.apply(lambda x: preprocess(str(x)))
reviews += len(romantic)

In [4]:
X = pd.Series()
y = np.empty(reviews)

In [5]:
index = 0
X = X.append(cheap)
y[index:len(cheap) + index] = 0
index += len(cheap)

X = X.append(expensive)
y[index:len(expensive) + index] = 1
index += len(expensive)

X = X.append(family)
y[index:(len(family) + index)] = 2
index += len(family)

X = X.append(friends)
y[index:len(friends) + index] = 3
index += len(friends)

X = X.append(kids)
y[index:len(kids) + index] = 4
index += len(kids)

X = X.append(romantic)
y[index:len(romantic) + index] = 5
index += len(romantic)

In [6]:
del cheap
del expensive
del family
del friends
del kids
del romantic

## Модель

In [75]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.pipeline import Pipeline
from sklearn.linear_model import (LinearRegression, Lasso, ElasticNet, Ridge,
                                  Perceptron, LogisticRegression,
                                  SGDClassifier)
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [76]:
classifiers =  [KNeighborsClassifier(3),
                SVC(kernel="linear", C=0.025),
                SVC(gamma=2, C=1),
                GaussianProcessClassifier(1.0 * RBF(1.0)),
                DecisionTreeClassifier(max_depth=5),
                RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
                MLPClassifier(alpha=1, max_iter=1000),
                AdaBoostClassifier(),
                GaussianNB(),
                QuadraticDiscriminantAnalysis()]

In [45]:
for min_df in range(1,5,1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)
    vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=300, min_df=2)
    vectorizer.fit(X_train)
    X_train = vectorizer.transform(X_train).toarray()
    X_test = vectorizer.transform(X_test).toarray()
    clf = MultinomialNB(alpha=0.01)
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test) * 100)

27.60577915376677
28.121775025799796
26.31578947368421
27.60577915376677


In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [78]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=300, min_df=2)
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

In [ ]:
for clf in classifiers:
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test) * 100)

13.777089783281735
